# Assignment 04 week 04 : Cassandra Python Programing

## lib imports

In [1]:
import pandas as pd 
from cassandra.cluster import Cluster

## Step 4: Connect to cassandra

In [2]:
session = Cluster().connect()

## Step 5:  Create keySpace

In [ ]:

keyspace_m14 = session.execute( """
CREATE KEYSPACE IF NOT EXISTS m14 
WITH REPLICATION = {
    'class' : 'SimpleStrategy',
    'replication_factor' : 1
};
""")

keyspace_m14

In [9]:
rows = session.execute('desc keyspaces')

for row in rows:
    print(row[0])

Row(keyspace_name='m14', type='keyspace', name='m14')
Row(keyspace_name='system', type='keyspace', name='system')
Row(keyspace_name='system_auth', type='keyspace', name='system_auth')
Row(keyspace_name='system_distributed', type='keyspace', name='system_distributed')
Row(keyspace_name='system_schema', type='keyspace', name='system_schema')
Row(keyspace_name='system_traces', type='keyspace', name='system_traces')
Row(keyspace_name='system_views', type='keyspace', name='system_views')
Row(keyspace_name='system_virtual_schema', type='keyspace', name='system_virtual_schema')


##  Step6: Create table: Inventory 

In [ ]:
create_table = """
CREATE TABLE IF NOT EXISTS  m14.inventory_by_sku (
sku int, 
name text, 
description text,
warehouse_num int,
PRIMARY KEY ((sku) )
);
"""
session.execute(create_table)

# Step7  A: Sample data for inventory table 

In [9]:
import pandas as pd 

sku = [1, 2, 3, 4, 5]
name = ['apple', 'ball', 'cat', 'dog', 'egg']
description = ['fruit', 'toy', 'animal', 'animal', 'meat']
warehouse_num = 123

mockup_data = [(sku[i], name[i], description[i], warehouse_num) for i in range(5)]

df = pd.DataFrame(mockup_data, columns=['sku', 'name', 'description', 'warehouse_num'])
df.loc[df['description'] == 'animal', 'warehouse_num'] = 456
df.head()

,sku,name,description,warehouse_num
0,1,apple,fruit,123
1,2,ball,toy,123
2,3,cat,animal,456
3,4,dog,animal,456
4,5,egg,meat,123


## Step7 B: Insert mockup data

In [13]:
for _, row in df.iterrows():
    insert_statement = f"INSERT INTO m14.inventory_by_sku (sku, name, description, warehouse_num) VALUES ({row['sku']}, '{row['name']}', '{row['description']}', {row['warehouse_num']});"
    session.execute(insert_statement)


## Step9: Read our inventory table 

In [16]:
read_table = """
SELECt * from m14.inventory_by_sku
WHERE warehouse_num = 123;
"""

rows = session.execute(read_table)

for row in rows: 
    print(row)

InvalidRequest: Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"

## Step8 : Create Index on Warehouse_num column

In [17]:
create_index = "CREATE  INDEX  warh_num_idx  on m14.inventory_by_sku(warehouse_num);"
_ = session.execute(create_index)

## Step 10: Search all products with warehouse_num 123 after index 

In [18]:
# Step run select again

rows = session.execute(read_table)
for row in rows:
    print(row)


Row(sku=1004, description='Action RPG', name='Assassins Creed Odyssey', warehouse_num=123)
Row(sku=1001, description='Action-adventure', name='Red Dead Redemption 2', warehouse_num=123)
Row(sku=1003, description='Open world', name='Grand Theft Auto V', warehouse_num=123)
Row(sku=1002, description='Action RPG', name='The Witcher 3: Wild Hunt', warehouse_num=123)
